In [ ]:
import pymysql
from openbb import obb

# 데이터베이스 연결 설정
def connect_to_mysql():
    connection = pymysql.connect(
        host='host',
        user='username',
        password='password',
        database='database'
    )
    return connection

# StockInfo 데이터 삽입 함수
def insert_stock_info(data):
    connection = None
    try:
        connection = connect_to_mysql()
        cursor = connection.cursor()

        # SQL 삽입 쿼리
        sql = """
        INSERT INTO stock_info (ticker, company_name)
        VALUES (%s, %s)
        ON DUPLICATE KEY UPDATE
        company_name = VALUES(company_name)
        """

        # 데이터베이스에 삽입
        cursor.executemany(sql, data)
        connection.commit()

        print(f"{cursor.rowcount} rows inserted/updated successfully.")
    except pymysql.MySQLError as e:
        print(f"MySQL Error: {e}")
    finally:
        if connection:
            connection.close()

# OpenBB에서 나스닥 티커와 회사 이름 가져오기
def fetch_nasdaq_tickers():
    nasdaq_stocks = obb.equity.search(exchange='intrinio')
    df = nasdaq_stocks.to_df()

    # symbol과 name 열만 선택
    company_tickers = df[['symbol', 'name']]
    return company_tickers.values.tolist()  # 리스트 형태로 변환

# Main 실행
if __name__ == "__main__":
    try:
        # 나스닥 데이터 가져오기
        nasdaq_data = fetch_nasdaq_tickers()
        print(f"Fetched {len(nasdaq_data)} records from NASDAQ.")

        # 데이터베이스에 삽입
        insert_stock_info(nasdaq_data)

    except Exception as e:
        print(f"Unexpected Error: {e}")
